In [ ]:
// Scala string style
import $ivy.`org.apache.spark::spark-sql:3.4.3`
import $ivy.`com.lihaoyi::os-lib:0.9.1`




In [3]:

// Add compiled classes to the classpath
import os._
interp.load.cp(Path("/home/kcat/Repos/newspipe/target/scala-2.13/classes"))
import org.apache.spark.sql._


import os._
import org.apache.spark.sql._

In [4]:

import notebookCode._

TrinoExploration.df


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/02 17:12:15 WARN Utils: Your hostname, Loki resolves to a loopback address: 127.0.1.1; using 192.168.100.36 instead (on interface enp7s0)
25/12/02 17:12:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/12/02 17:12:19 INFO SparkContext: Running Spark version 3.4.3
25/12/02 17:12:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/12/02 17:12:19 INFO ResourceUtils: ==============================================================
25/12/02 17:12:19 INFO ResourceUtils: No custom resources configured for spark.driver.
25/12/02 17:12:19 INFO ResourceUtils: ==============================================================
25/12/02 17:12:19 INFO SparkContext: Submitted application: TrinoNotebook
25/12/02 17:12:19 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: core

java.io.FileNotFoundException: .env (No such file or directory)